In [27]:
import pandas as pd
import math 
import numpy as np
from numpy.linalg import norm
from scipy import stats
from obspy.geodetics import degrees2kilometers
from time import time_ns

In [3]:
def init_dataframe(path):
    df = pd.read_csv(path, sep=',')
    return df

In [4]:
ref1 = init_dataframe('data/ref1.csv')
ref2 = init_dataframe('data/ref2.csv')

In [6]:
ref2.head(2)

,LAT,LON,GSPEED,CRS,NLAT,NLON,NCRS
0,48.050451,17.569713,0.82,209.35145,48.05045,17.569712,208.75295
1,48.050450,17.569712,0.89,208.75295,48.05045,17.569712,208.97508


In [37]:
lap = init_dataframe('data/laps/lap1-26.csv')
lap.head(2)

,LAT,LON,GSPEED,CRS,NLAT,NLON,NCRS,CMD
0,48.049216,17.567882,21.44,212.85306,48.049200,17.567865,214.09550,4.541553
1,48.049200,17.567865,21.59,214.09550,48.049184,17.567848,214.79764,3.492088


In [16]:
lap[['LAT','LON']].head(2)

,LAT,LON
0,48.049238,17.567861
1,48.049235,17.567858


In [38]:
len(lap)

556

In [33]:
def line_length(x1,y1,x2,y2):
    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

def find_closest_point(point, lap):
    minIndex = 0
    minLength = math.inf
    for i in lap.index:
        lat = lap.loc[i].LAT
        lon = lap.loc[i].LON
        length = line_length(lat, lon, point.LAT, point.LON)
        if length < minLength:
            minIndex = i
            minLength = length

    return minIndex

def find_angle_between_vectors(vector_A, vector_B):
    unit_vector_A = vector_A / np.linalg.norm(vector_A) 
    unit_vector_B = vector_B / np.linalg.norm(vector_B)
    dot_product = np.dot(unit_vector_A, unit_vector_B)
    return np.arccos(dot_product)                       #return angle in radians 

def create_vector(point_A, point_B):
    return [point_B.LAT - point_A.LAT, point_B.LON - point_A.LON]

# Perdendicular from p1 to line (p2,p3)
def shortest_distance(p1,p2,p3):  
    dist = norm(np.cross(p2-p3, p3-p1))/norm(p3-p2)
    return dist

def shortest_distance2(x1, y1, a, b, c):  
    perpendicular = abs((a * x1 + b * y1 + c)) / (math.sqrt(a ** 2 + b ** 2)) 
    return perpendicular

def find_shortest_distance(p1, p2, p3):
    x = np.array([p1.LAT, p1.LON])
    y = np.array([p2.LAT, p2.LON])
    z = np.array([p3.LAT, p3.LON])
    return shortest_distance(x,y,z)

def line_length(x1,y1,x2,y2):
    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

In [26]:
x1 = 10
y1 = 20
a = 2
b = -1
c = 0
shortest_distance(x1, y1, a, b, c)   

0.0

In [36]:
sum_of_distances = 0
distances = []

for i in lap.index:
    point = lap.loc[i]

    closest_index = find_closest_point(point, ref1)
    closest_point = ref1.loc[closest_index]

    neighbor_i = len(ref1)-1 if closest_index == 0 else closest_index-1
    neighbor1 = ref1.loc[neighbor_i]
    neighbor_i = 0 if len(ref1) == closest_index+1 else closest_index+1
    neighbor2 = ref1.loc[neighbor_i]

    v1 = create_vector(closest_point, point)
    v2 = create_vector(closest_point, neighbor1)
    v3 = create_vector(closest_point, neighbor2)

    angle1 = find_angle_between_vectors(v1,v2)
    angle2 = find_angle_between_vectors(v1,v3)

    degrees90 = math.pi/2
    min_dist = -1
    if angle1 > degrees90 and angle2 > degrees90:
        min_dist = line_length(point.LAT, point.LON, closest_point.LAT, closest_point.LON)
    elif angle1 < degrees90 and angle2 < degrees90:
        dist1 = find_shortest_distance(point, closest_point, neighbor1)
        dist2 = find_shortest_distance(point, closest_point, neighbor2)
        min_dist = dist1 if dist1 <= dist2 else dist2
    elif angle1 < degrees90:
        min_dist = find_shortest_distance(point, closest_point, neighbor1)
    elif angle2 < degrees90:
        min_dist = find_shortest_distance(point, closest_point, neighbor2)

    if min_dist == -1:
        print ('ERROR: Could not find distance')       
        print("Indices: {} {}\nAngles: {} {}".format(i, closest_index, angle1, angle2))
    elif math.isnan(min_dist):
        print("NAN value!!!\nIndices: {} {}\nAngles: {} {}".format(i, closest_index, angle1, angle2))
    elif min_dist < 0:
        print("Negative value!!!\nIndices: {} {}\nAngles: {} {}".format(i, closest_index, angle1, angle2))
    else:
        min_dist = degrees2kilometers(min_dist) * 100000    # in centimeters
        sum_of_distances += min_dist
        distances.append(min_dist)

diff_df = pd.DataFrame({"Distances":distances})

print ("Sum is {}cm".format(sum_of_distances))
print ("Average distance from reference lap is {}cm".format(sum_of_distances/len(lap)))

# with open('average_perpendiculars.txt', 'a') as file:
#     file.write('{} cm\n'.format(sum_of_distances/len(lap)))


    

Sum is 106441.4951730276cm
Average distance from reference lap is 102.54479303759884cm


In [42]:
diff_df.sum()/len(diff_df)

Distances    102.544793
dtype: float64

In [100]:
lap1_0 = pd.read_csv('data/1-0.csv',index_col=0)

In [99]:
lap1_0.head(2)

,lap1-0.csv,lap1-0.csv (from view of ref lap)
0,44.521917,9.435202
1,59.509375,8.468353


In [109]:
lap1_0['lap1-0.csv'].max(), \
lap1_0['lap1-0.csv (from view of ref lap)'].max()

(613.21327076199998, 612.71149384800003)

In [110]:
lap1_0['lap1-0.csv'].min(), \
lap1_0['lap1-0.csv (from view of ref lap)'].min()

(0.023808518632999997, 0.103757408482)

In [113]:
lap1_0['lap1-0.csv'].mean(), \
lap1_0['lap1-0.csv (from view of ref lap)'].mean()

(88.010888204534723, 115.55902806110871)